In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from matplotlib.colors import LinearSegmentedColormap, to_hex
import ipywidgets as widgets
from scipy.stats import skew

# Display options:
pd.set_option("display.width", 1200)
pd.set_option("display.max_columns", 300)
pd.set_option("display.max_rows", 300)

In [2]:
# Read the data:
df_videos = pd.read_csv("data/videos_data.csv", sep = ";")

In [3]:
df_videos

,channel_title,channel_id,video_title,video_id,video_upload_date,views,likes,dislikes,comments,age_days,likes_dislikes_ratio,comments_views_ratio,mean_views_day
0,3Blue1Brown,UCYO_jab_esuFRV4b17AJtAw,How a Mandelbrot set arises from Newton’s work,LqbZpur38nw,2021-10-15 16:41:50+00:00,568689,26102,146,1216,8.0,178.780822,0.002138,71086.125000
1,3Blue1Brown,UCYO_jab_esuFRV4b17AJtAw,Newton's Fractal (which Newton knew nothing ab...,-RdOwhmqP5s,2021-10-07 02:19:39+00:00,1195545,61723,284,2810,16.0,217.334507,0.002350,74721.562500
2,3Blue1Brown,UCYO_jab_esuFRV4b17AJtAw,The Summer of Math Exposition,ojjzXyQCzso,2021-07-16 15:37:16+00:00,610550,29300,215,1719,99.0,136.279070,0.002815,6167.171717
3,3Blue1Brown,UCYO_jab_esuFRV4b17AJtAw,A quick trick for computing eigenvalues | Chap...,e50Bj7jn9IQ,2021-05-07 19:01:16+00:00,421914,17032,145,1146,169.0,117.462069,0.002716,2496.532544
4,3Blue1Brown,UCYO_jab_esuFRV4b17AJtAw,How (and why) to raise e to the power of a mat...,O85OWBJ2ayo,2021-04-01 06:38:35+00:00,1161654,46425,296,2586,205.0,156.841216,0.002226,5666.604878
...,...,...,...,...,...,...,...,...,...,...,...,...,...
109841,zefrank1,UCVpankR4HtoAVtYnFDUieYA,HardTimes :: Art Hour,C6hpMftAIs0,2009-06-13 18:32:12+00:00,172742,2155,31,136,4515.0,69.516129,0.000787,38.259579
109842,zefrank1,UCVpankR4HtoAVtYnFDUieYA,HardTimes :: Hole,LPfcGXMpKds,2009-06-06 22:16:56+00:00,90817,1793,23,75,4521.0,77.956522,0.000826,20.087812
109843,zefrank1,UCVpankR4HtoAVtYnFDUieYA,HardTimes :: Affordable Button,h3lvSflNixI,2009-05-30 00:13:46+00:00,103164,1566,47,18,4529.0,33.319149,0.000174,22.778538
109844,zefrank1,UCVpankR4HtoAVtYnFDUieYA,HardTimes :: Outsource,zxIbQ6ZG4OI,2009-05-17 00:50:31+00:00,78211,1620,20,87,4542.0,81.000000,0.001112,17.219507


In [40]:
df_videos = df_videos.astype({
    "views": "float64",
    "likes": "float64",
    "dislikes": "float64",
    "comments": "float64"
})

In [42]:
df_videos.dtypes

channel_title            object
channel_id               object
video_title              object
video_id                 object
video_upload_date        object
views                   float64
likes                   float64
dislikes                float64
comments                float64
age_days                float64
likes_dislikes_ratio    float64
comments_views_ratio    float64
mean_views_day          float64
dtype: object

In [43]:
df_videos.describe()

,views,likes,dislikes,comments,age_days,likes_dislikes_ratio,comments_views_ratio,mean_views_day
count,1.098450e+05,1.098450e+05,1.098450e+05,109845.000000,109845.000000,109845.000000,109845.000000,1.098450e+05
mean,6.610346e+05,1.330990e+04,4.659082e+02,769.016232,1832.940835,56.082814,0.003138,9.710776e+02
std,3.377005e+06,4.733269e+04,7.937326e+03,3076.703703,1226.739195,78.498807,0.006612,8.986263e+03
min,1.000000e+00,0.000000e+00,1.000000e+00,0.000000,2.000000,0.000000,0.000000,7.012623e-04
25%,1.623000e+04,2.270000e+02,1.000000e+01,24.000000,883.000000,12.797153,0.000647,9.232263e+00
50%,6.793200e+04,1.356000e+03,4.000000e+01,120.000000,1648.000000,32.076923,0.001555,5.030630e+01
75%,2.880430e+05,7.036000e+03,1.690000e+02,509.000000,2570.000000,68.629630,0.003415,2.946695e+02
max,3.244799e+08,3.061149e+06,2.384345e+06,298356.000000,5732.000000,2635.000000,0.440313,1.030206e+06


In [9]:
# Removed the channel "arewethereyet" because it has been left with only one video after 
# removing NaN, for being so damn good that is has 75% of its videos with 0 dislikes.
df_videos = df_videos[df_videos["channel_title"] != "arewethereyet"]
df_videos = df_videos.reset_index(drop = True)

In [14]:
# Scatter of Mean views per day and Likes-disikes ratio:
x_var_name = "Likes-dislikes ratio"
y_var_name = "Mean views per day"
color_var_name = "Age (days)"
x_var = "likes_dislikes_ratio"
y_var = "mean_views_day"
color_var = "age_days"

channel = "Bob Ross"
df_plot = df_videos[df_videos["channel_title"] == channel]

n_colors = 100
my_colors = ["#000000", "#E008F8", "#F81D08", "#F88A08", "#F7FE04"]
cmap = LinearSegmentedColormap.from_list("my_palette", my_colors)
my_palette = [to_hex(j) for j in  [cmap(i/n_colors) for i in np.array(range(n_colors))]]

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x = df_plot[x_var],
        y = df_plot[y_var],
        mode = "markers",
        marker = {
            "size": 7,
            "color": df_plot[color_var],
            "colorscale": my_palette,
            "showscale": True,
            "colorbar": {
                "title": "<b>" + color_var_name + "</b>"
            }
        },        
        text = df_plot["video_title"],
        customdata = df_plot[color_var],
        hovertemplate = "<b>" + x_var_name + ": %{x:}<br>" +
                         y_var_name + ": %{y:}<br>" +
                         color_var_name + ": %{customdata}<br>" +
                         "Video: %{text}</b><extra></extra>"
    )
)
fig.update_layout(
    title = "<b>" + channel + "</b>" ,
    xaxis_title = "<b>" + x_var_name + "</b>" ,
    yaxis_title = "<b>" + y_var_name + "</b>",
    font = dict(
        size = 18
    ),
    plot_bgcolor = "white",
    hoverlabel = dict(
        font_size = 20,
        font_family = "Rockwell"
    )
)

In [78]:
# Scatter of likes and dislikes:
x_var_name = "Ages (days)"
y_var_name = "Comments-views ratio"
color_var_name = "Mean views per day"
size_var_name = "Likes-dislikes ratio"
x_var = "age_days"
y_var = "comments_views_ratio"
color_var = "mean_views_day"
size_var = "likes_dislikes_ratio"

channel = "Kurzgesagt – In a Nutshell"
df_plot = df_videos[df_videos["channel_title"] == channel]

n_colors = 100
my_colors = ["#000000", "#E008F8", "#F81D08", "#F88A08", "#F7FE04"]
cmap = LinearSegmentedColormap.from_list("my_palette", my_colors)
my_palette = [to_hex(j) for j in  [cmap(i/n_colors) for i in np.array(range(n_colors))]]

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x = df_plot[x_var],
        y = df_plot[y_var],
        mode = "markers",
        marker = {
            "color": df_plot[color_var],
            "colorscale": my_palette,
            "showscale": True,
            "colorbar": {
                "title": "<b>" + color_var_name + "</b>"
            },
            "size": df_plot[size_var],
            "opacity": 0.9,
            "sizemode": "area",
            "sizeref": 2.*max(df_plot[size_var])/(40.**2),
            "sizemin": 4            
        },
        text = df_plot["video_title"],
        customdata = df_plot[[color_var, size_var]],
        hovertemplate = "<b>" + x_var_name + ": %{x:.0f}<br>" +
                         y_var_name + ": %{y:.2g}<br>" +
                         color_var_name + ": %{customdata[0]:.0f}<br>" +
                         size_var_name + ": %{customdata[1]:.3f}<br>" +
                         "Video: %{text}</b><extra></extra>"
    )
)
fig.update_layout(
    title = "<b>" + channel + "</b>" ,
    xaxis_title = "<b>" + x_var_name + "</b>" ,
    yaxis_title = "<b>" + y_var_name + "</b>",
    font = dict(
        size = 18
    ),
    plot_bgcolor = "white",
    hoverlabel = dict(
        font_size = 20,
        font_family = "Rockwell"
    )
)

In [ ]:
### Dynamic plots (choosing the channel, the variables and some filters)

In [143]:
# Variables names:
vars_names = {
    "Views": "views",
    "Likes": "likes",
    "Dislikes": "dislikes",
    "Comments": "comments",
    "Likes-dislikes ratio": "likes_dislikes_ratio",
    "Age (days)": "age_days",
    "Comment-views ratio": "comments_views_ratio",
    "Mean views per day": "mean_views_day"
}

# Options:
opts_channel = np.sort(df_videos["channel_title"].unique()).tolist()
opts_vars = list(vars_names.keys())
opts_vars = [i for i in vars_names.keys()]
opts_vars.sort()

# Custom hovers:
custom_vars = list(vars_names.values())
custom_template = "<b>Video title: %{text}<br>"
for i in range(0, len(custom_vars) - 1):
    custom_template += list(vars_names.keys())[i] + ": %{customdata[" + str(i) + "]:}<br>"


In [144]:
custom_vars

['views',
 'likes',
 'dislikes',
 'comments',
 'likes_dislikes_ratio',
 'age_days',
 'comments_views_ratio',
 'mean_views_day']

In [145]:
custom_template

'<b>Video title: %{text}<br>Views: %{customdata[0]:}<br>Likes: %{customdata[1]:}<br>Dislikes: %{customdata[2]:}<br>Comments: %{customdata[3]:}<br>Likes-dislikes ratio: %{customdata[4]:}<br>Age (days): %{customdata[5]:}<br>Comment-views ratio: %{customdata[6]:}<br>'

In [146]:
df_videos[custom_vars]

,views,likes,dislikes,comments,likes_dislikes_ratio,age_days,comments_views_ratio,mean_views_day
0,568689.0,26102.0,146.0,1216.0,178.780822,8.0,0.002138,71086.125000
1,1195545.0,61723.0,284.0,2810.0,217.334507,16.0,0.002350,74721.562500
2,610550.0,29300.0,215.0,1719.0,136.279070,99.0,0.002815,6167.171717
3,421914.0,17032.0,145.0,1146.0,117.462069,169.0,0.002716,2496.532544
4,1161654.0,46425.0,296.0,2586.0,156.841216,205.0,0.002226,5666.604878
...,...,...,...,...,...,...,...,...
109840,172742.0,2155.0,31.0,136.0,69.516129,4515.0,0.000787,38.259579
109841,90817.0,1793.0,23.0,75.0,77.956522,4521.0,0.000826,20.087812
109842,103164.0,1566.0,47.0,18.0,33.319149,4529.0,0.000174,22.778538
109843,78211.0,1620.0,20.0,87.0,81.000000,4542.0,0.001112,17.219507


In [61]:
### 1D histogram

# Variable and filters widgets:
input_xvar = widgets.Dropdown(
    description = "Variable: ",
    options = list(vars_names.keys()),
    value = list(vars_names.keys())[0]
)
input_channels = widgets.Dropdown(
    description = "Channel: ",
    options = opts_channel,
    value = opts_channel[0]
)
input_bins = widgets.IntSlider(
    description = "Bins: ",
    min = 10,
    max = 1000,
    step = 10,
    value = 200
)
input_xmin = widgets.FloatText(
    value = np.min(df_videos[vars_names[input_xvar.value]]),
    description = "Min: "
)
input_xmax = widgets.FloatText(
    value = np.max(df_videos[vars_names[input_xvar.value]]),
    description = "Max: "
)

# Statistics in the title:
def plot_title(x):
    title = "Filtered sample statistics<br>" +\
            "<b style = 'color: #900c3f'>Mean</b>: " + f"{np.mean(x):.7g}      " +\
            "<b style = 'color: #ffc300'>Median</b>: " + f"{np.median(x):.7g}      " +\
            "<b>Standard deviation</b>: " + f"{np.std(x):.7g}      " +\
            "<b>Skewness</b>: " + f"{skew(x):.3g}"
    return(title)

# Mean and median lines:
def vert_lines(x):
    vertical_lines = [
        {
            'line': {
                'color': '#900c3f',
                'dash': 'dash',
                'width': 2
            },
            'type': 'line',
            'x0': np.mean(x),
            'x1': np.mean(x),
            'xref': 'x',
            'y0': 0,
            'y1': 1,
            'yref': 'paper'
        },
        {
            'line': {
                'color': '#ffc300',
                'dash': 'dash',
                'width': 2
            },
            'type': 'line',
            'x0': np.median(x),
            'x1': np.median(x),
            'xref': 'x',
            'y0': 0,
            'y1': 1,
            'yref': 'paper'
        }
    ]
    return(vertical_lines)

# Initialize the figure:
x_init = df_videos.loc[df_videos["channel_title"] == opts_channel[0], vars_names[input_xvar.value]]
x_mean = np.mean(x_init)
x_median = np.median(x_init)
fig = go.FigureWidget(
    data = [
        go.Histogram(
            x = x_init,
            histfunc = "count",
            nbinsx = input_bins.value,
            marker_color = "#00baad",
            opacity = 0.9
        )
    ],
    layout = go.Layout(
        title = plot_title(x_init),
        xaxis_title = "<b>" + input_xvar.value + "</b>",
        yaxis_title = "<b>Counts</b>",
        font = dict(
            size = 18
        ),
        showlegend = False,
        plot_bgcolor = "white",
        hoverlabel = dict(
            font_size = 18,
            font_family = "Rockwell"
        ),
        margin = dict(
            l = 20,
            r = 20,
            t = 100,
            b = 20
        ),
        height = 600,
        shapes = vert_lines(x_init)
    )
)

# Filter and update function:
def filtering(chosen_xvar, chosen_channel, chosen_bins, chosen_xmin, chosen_xmax):
    # Filter by channel:
    df_filtered = df_videos.copy()[df_videos["channel_title"] == chosen_channel]
    
    # Variable:
    x_vals = df_filtered[vars_names[chosen_xvar]]

    # Filter the range:
    if chosen_xmin >= chosen_xmax or chosen_xmax < np.min(x_vals) or chosen_xmin > np.max(x_vals):
        pass
    else:
        if chosen_xmin < np.min(x_vals):
            pass
        else:
            x_vals = x_vals[x_vals > chosen_xmin]
        if chosen_xmax > np.max(x_vals):
            pass
        else:
            x_vals = x_vals[x_vals < chosen_xmax]
    
    # Drop the nan:
    x_vals = x_vals.dropna()

    # Update the figure:
    with fig.batch_update():
        fig.data[0].x = x_vals
        fig.data[0].nbinsx = chosen_bins
        fig.layout.xaxis.title = "<b>" + chosen_xvar + "</b>"
        fig.layout.title = plot_title(x_vals)
        fig.layout.shapes = vert_lines(x_vals)

# Event handlers:
def eventhandler_xvar(change):
    filtering(chosen_xvar = change.new,
              chosen_channel = input_channels.value,
              chosen_bins = input_bins.value,
              chosen_xmin = input_xmin.value,
              chosen_xmax = input_xmax.value)
def eventhandler_channels(change):
    filtering(chosen_xvar = input_xvar.value,
              chosen_channel = change.new,
              chosen_bins = input_bins.value,
              chosen_xmin = input_xmin.value,
              chosen_xmax = input_xmax.value)
def eventhandler_bins(change):
    filtering(chosen_xvar = input_xvar.value,
              chosen_channel = input_channels.value,
              chosen_bins = change.new,
              chosen_xmin = input_xmin.value,
              chosen_xmax = input_xmax.value)
def eventhandler_xmin(change):
    filtering(chosen_xvar = input_xvar.value,
              chosen_channel = input_channels.value,
              chosen_bins = input_bins.value,
              chosen_xmin = change.new,
              chosen_xmax = input_xmax.value)
def eventhandler_xmax(change):
    filtering(chosen_xvar = input_xvar.value,
              chosen_channel = input_channels.value,
              chosen_bins = input_bins.value,
              chosen_xmin = input_xmin.value,
              chosen_xmax = change.new)

# Observes:
input_xvar.observe(eventhandler_xvar,
                   names = "value")
input_channels.observe(eventhandler_channels,
                       names = "value")
input_bins.observe(eventhandler_bins,
                   names = "value")
input_xmin.observe(eventhandler_xmin,
                   names = "value")
input_xmax.observe(eventhandler_xmax,
                   names = "value")

# Row of filters:
row_filters = widgets.HBox(
    [
        input_xvar,
        input_channels,
        input_bins,
        input_xmin,
        input_xmax
    ]
)

# Main box:
widgets.VBox(
    [
        row_filters,
        fig
    ]
)

In [148]:
### Scatter with colors

# Variables and filters widgets:
input_xvar = widgets.Dropdown(
    description = "x: ",
    options = list(vars_names.keys()),
    value = list(vars_names.keys())[0]
)
input_yvar = widgets.Dropdown(
    description = "y: ",
    options = list(vars_names.keys()),
    value = list(vars_names.keys())[1]
)
input_cvar = widgets.Dropdown(
    description = "color: ",
    options = list(vars_names.keys()),
    value = list(vars_names.keys())[2]
)
input_channels = widgets.Dropdown(
    description = "Channel: ",
    options = opts_channel,
    value = opts_channel[0]
)

# Initialize the figure:
n_colors = 100
my_colors = ["#000000", "#E008F8", "#F81D08", "#F88A08", "#F7FE04"]
cmap = LinearSegmentedColormap.from_list("my_palette", my_colors)
my_palette = [to_hex(j) for j in  [cmap(i/n_colors) for i in np.array(range(n_colors))]]

df_plot = df_videos[df_videos["channel_title"] == opts_channel[0]]
x_init = df_plot[vars_names[input_xvar.value]]
y_init = df_plot[vars_names[input_yvar.value]]
c_init = df_plot[vars_names[input_cvar.value]]

fig = go.FigureWidget(
    data = [
        go.Scatter(
            x = x_init,
            y = y_init,
            mode = "markers",
            marker = {
                "size": 7,
                "color": c_init,
                "colorscale": my_palette,
                "showscale": True,
                "colorbar": {
                    "title": "<b>" + input_cvar.value + "</b>"
                }
            },
            text = df_plot["video_title"],
            customdata = df_plot[custom_vars],
            hovertemplate = custom_template
        )
    ],
    layout = go.Layout(
        xaxis_title = "<b>" + input_xvar.value + "</b>",
        yaxis_title = "<b>" + input_yvar.value + "</b>",
        font = dict(
            size = 18
        ),
        showlegend = False,
        plot_bgcolor = "white",
        hoverlabel = dict(
            font_size = 18,
            font_family = "Rockwell"
        ),
        margin = dict(
            l = 20,
            r = 20,
            t = 20,
            b = 20
        ),
        height = 600
    )
)

# Filter and update function:
def filtering(chosen_xvar, chosen_yvar, chosen_cvar, chosen_channel):
    # Filter by channel:
    if chosen_channel == "All channels":
        df_filtered = df_videos.copy()
    else:
        df_filtered = df_videos.copy()[df_videos["channel_title"] == chosen_channel]
    
    # Variables:
    x_vals = df_filtered[vars_names[chosen_xvar]]
    y_vals = df_filtered[vars_names[chosen_yvar]]
    c_vals = df_filtered[vars_names[chosen_cvar]]

    # Update the figure:
    with fig.batch_update():
        fig.data[0].x = x_vals
        fig.data[0].y = y_vals
        fig.layout.xaxis.title = "<b>" + chosen_xvar + "</b>"
        fig.layout.yaxis.title = "<b>" + chosen_yvar + "</b>"
        fig.data[0].marker.color = c_vals
        fig.data[0].marker.colorbar.title = "<b>" + chosen_cvar + "</b>"
        fig.data[0].text = df_filtered["video_title"]
        fig.data[0].customdata = df_filtered[custom_vars]



# Event handlers:
def eventhandler_xvar(change):
    filtering(chosen_xvar = change.new,
              chosen_yvar = input_yvar.value,
              chosen_cvar = input_cvar.value,
              chosen_channel = input_channels.value)
def eventhandler_yvar(change):
    filtering(chosen_xvar = input_xvar.value,
              chosen_yvar = change.new,
              chosen_cvar = input_cvar.value,
              chosen_channel = input_channels.value)
def eventhandler_cvar(change):
    filtering(chosen_xvar = input_xvar.value,
              chosen_yvar = input_yvar.value,
              chosen_cvar = change.new,
              chosen_channel = input_channels.value)
def eventhandler_channels(change):
    filtering(chosen_xvar = input_xvar.value,
              chosen_yvar = input_yvar.value,
              chosen_cvar = input_cvar.value,
              chosen_channel = change.new)

# Observes:
input_xvar.observe(eventhandler_xvar,
                   names = "value")
input_yvar.observe(eventhandler_yvar,
                   names = "value")
input_cvar.observe(eventhandler_cvar,
                   names = "value")
input_channels.observe(eventhandler_channels,
                       names = "value")

# Row of filters:
row_filters = widgets.HBox(
    [
        input_xvar,
        input_yvar,
        input_cvar,
        input_channels
    ]
)

# Main box:
widgets.VBox(
    [
        row_filters,
        fig
    ]
)

In [7]:
# Barplot of number of videos by channel:

df = df_videos['channel_title'].value_counts()

In [8]:
df

BBC                                       13180
CNBC                                      10779
Classic Mr Bean                            9332
Netflix                                    4963
Business Insider                           4795
Adam Savage’s Tested                       4451
NASA                                       4232
TED                                        3693
SpaceToday                                 3416
European Space Agency, ESA                 3042
SciShow                                    2792
Scott Manley                               2398
Canal do Schwarza                          2361
Disney Parks                               2301
Porta dos Fundos                           1735
Jacques Slade                              1707
Meteoro Brasil                             1535
Guinness World Records                     1449
Parafernalha                               1191
Aviões e Músicas com Lito Sousa            1088
Fabio Chaves                            

In [ ]:
# Bubble with colors:

In [ ]:
# 2D density for 2 channels:

In [ ]:
# Correlation matrix continuous: